In [38]:
import pandas as pd
import numpy as np
from datetime import datetime

file_path = '/Users/HM/Desktop/Code/FOIL_DA_Task-10-2024/data/Online_Retail.xlsx'
xls_data = pd.read_excel(file_path)

def load_data(file_path):
    file_path = '/Users/HM/Desktop/Code/FOIL_DA_Task-10-2024/data/Online_Retail.xlsx'
    xls_data = pd.read_excel(file_path)

    xls_data = xls_data.sort_values(by='InvoiceDate').reset_index(drop=True)

    return xls_data.shape, xls_data.head(5)

Some price data is negative, while quantities for certain items are also negative implying outflow.

A large number of CustomerIDs are missing.

Some items seem to have been mislabeled as other items.

Entries in the Description column are in combination upper- and lower-case, while item names are entirely in upper-case.

In [39]:
def empty_values(file_path):
    
    file_path = file_path = '/Users/HM/Desktop/Code/FOIL_DA_Task-10-2024/data/Online_Retail.xlsx'
    
    xls_data = pd.read_excel(file_path)

    cols = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']
    
    empty_values_count = {}
    
    for col in cols:
        if xls_data[col].dtype == 'object':
            empty_values = xls_data[xls_data[col].isnull() | xls_data[col].str.strip() == ""]
        else:
            empty_values = xls_data[xls_data[col].isnull()]

        empty_values_count[col] = len(empty_values)

        if len(empty_values) > 0:
            print(f"Column {col} has {len(empty_values)} missing or empty values")
        else:
            print(f"No empty or missing values found in {col} column")
            
    return empty_values_count

Filling empty Customer IDs with -1

In [40]:
def fill_missing_cust_id(file_path):
    file_path = 'data/Online_Retail.xlsx'
    xls_data = pd.read_excel(file_path)
    prior_missing_cust_ids= xls_data['CustomerID'].isnull().sum()
    xls_data['CustomerID'] = xls_data['CustomerID'].fillna(-1)

    post_missing_cust_ids = xls_data['CustomerID'].isnull().sum()

    return prior_missing_cust_ids, post_missing_cust_ids

In [41]:
print(load_data(file_path))

((541909, 8),   InvoiceNo StockCode                          Description  Quantity  \
0    536365     71053                  WHITE METAL LANTERN         6   
1    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
2    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
3    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
4    536365     22752         SET 7 BABUSHKA NESTING BOXES         2   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
1 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
2 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       7.65     17850.0  United Kingdom  )


In [42]:
print(empty_values(file_path))

No empty or missing values found in InvoiceNo column
No empty or missing values found in StockCode column
No empty or missing values found in Description column
No empty or missing values found in Quantity column
No empty or missing values found in InvoiceDate column
No empty or missing values found in UnitPrice column
Column CustomerID has 135080 missing or empty values
No empty or missing values found in Country column
{'InvoiceNo': 0, 'StockCode': 0, 'Description': 0, 'Quantity': 0, 'InvoiceDate': 0, 'UnitPrice': 0, 'CustomerID': 135080, 'Country': 0}
